## 경사 하강법 구현

* 그래디언트 수동 구현
* 텐서플로의 자동 미분 기능을 이용한 자동 계싼
* 텐서플로에 내장된 [옵티마이저](https://goo.gl/e7ptdr)(tf.train.Optimzer)

In [1]:
import numpy as np
from sklearn.datasets import fetch_california_housing
import tensorflow as tf

In [16]:
# 데이터 셋을 추출하고 
# 모든 훈련 샘플에 편향에 대한 입력 특성(x0 = 1)을 추가
housing = fetch_california_housing()
m, n = housing.data.shape

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled_housing_data = scaler.fit_transform(housing.data)

housing_data_plus_bias = np.c_[np.ones((m, 1)), scaled_housing_data]   # (m, 1)행렬을 만든뒤 housing데이터의 값을 채워넣음.
housing_data_plus_bias

# housing_data_plus_bias = np.c_[np.ones((m, 1)), housing.data]   # (m, 1)행렬을 만든뒤 housing데이터의 값을 채워넣음.
# housing_data_plus_bias

array([[ 1.        ,  2.34476576,  0.98214266, ..., -0.04959654,
         1.05254828, -1.32783522],
       [ 1.        ,  2.33223796, -0.60701891, ..., -0.09251223,
         1.04318455, -1.32284391],
       [ 1.        ,  1.7826994 ,  1.85618152, ..., -0.02584253,
         1.03850269, -1.33282653],
       ...,
       [ 1.        , -1.14259331, -0.92485123, ..., -0.0717345 ,
         1.77823747, -0.8237132 ],
       [ 1.        , -1.05458292, -0.84539315, ..., -0.09122515,
         1.77823747, -0.87362627],
       [ 1.        , -0.78012947, -1.00430931, ..., -0.04368215,
         1.75014627, -0.83369581]])

### 그래디언트 수동 구현
* random_uniform() 함수는 난수를 담은 텐서를 생성하는 노드를 그래프에 생성한다. numpy의 rand() 함수처럼 크기와 난수의 범위를 입력받는다.
* assign() 함수는 변수에 새로운 값을 할당하는 노드를 생성한다.
* 반복 루프는 훈련 단계를 계속 반복해서 실행하고 100번 반복마다 현재의 평균 제곱 에러(코드에서 mse 변수)를 출력

In [19]:
n_epochs = 1000
learning_rate = 0.01

X = tf.constant(housing_data_plus_bias, dtype=tf.float32, name="X")             # (20640, 9)
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name='y')      # (20640, 1)
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0), name='theta')     # (9, 1)
y_pred = tf.matmul(X, theta, name="predictions")

error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name='mse')
gradients = 2/m * tf.matmul(tf.transpose(X), error)
training_op = tf.assign(theta, theta - learning_rate * gradients)  # theta 라는 변수에 아래 연산을 할당..

In [20]:
init = tf.global_variables_initializer()  # 변수 초기화

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(1000):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE", mse.eval())
        sess.run(training_op)
    
    best_theta = theta.eval()
    print(best_theta)

Epoch 0 MSE 4.260892
Epoch 100 MSE 0.8090007
Epoch 200 MSE 0.6939066
Epoch 300 MSE 0.6456543
Epoch 400 MSE 0.61179495
Epoch 500 MSE 0.5874149
Epoch 600 MSE 0.5698319
Epoch 700 MSE 0.55714893
Epoch 800 MSE 0.5480005
Epoch 900 MSE 0.5414016
[[ 2.0685523 ]
 [ 0.7647554 ]
 [ 0.15040347]
 [-0.06102888]
 [ 0.10055514]
 [ 0.00785305]
 [-0.04062066]
 [-0.7300592 ]
 [-0.6892007 ]]


### 자동 미분 사용
추후 기재..

In [13]:
gradients = tf.gradients(mse, [theta])[0]

### 옵티마이저 사용

In [14]:
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

In [15]:
optimizer = tf.train.MomentumOptimizer(learning_rate=learning_rate, momentum=0.9)